In [1]:
import cv2 as cv
import os
import numpy as np


In [2]:
def preprocess_image_level(img_path):
    img = cv.imread(img_path)

    resized_img = cv.resize(img, (640, 640))

    contrast = cv.convertScaleAbs(resized_img, alpha=0.9, beta=0)
    grey = cv.cvtColor(contrast, cv.COLOR_BGR2GRAY)
    # cv.imshow("Blur", denoised_img)
    noise_reduction = cv.GaussianBlur(grey, (9, 9), 0)
    adaptive_thresh = cv.adaptiveThreshold(
        noise_reduction,
        255,
        cv.CALIB_CB_ADAPTIVE_THRESH,
        cv.THRESH_BINARY_INV,
        23,
        5,
    )

    lap = cv.Laplacian(adaptive_thresh, cv.CV_64F)
    lap = np.uint8(np.absolute(lap))

    kernel = np.ones((5, 5), np.uint8)
    dilate = cv.dilate(lap, kernel, iterations=1)

    erosion = cv.erode(dilate, (7, 7), iterations=4)

    normalize = cv.normalize(
        erosion,
        None,
        alpha=0,
        beta=300,
        norm_type=cv.NORM_MINMAX,
        dtype=cv.WARP_INVERSE_MAP,
    )

    mask = cv.bitwise_not(normalize)
    croped = mask[0:700, 15:640]

    return croped

In [3]:
img_path = "/home/uwu/Desktop/object_detection/preview/yolo/1f9e8765-44db-4303-b12a-d6b30c9cc5f9.png"

In [4]:
mask = preprocess_image_level(img_path)

cv.imshow("test", mask)
cv.waitKey(0)
cv.destroyAllWindows()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
